In [1]:
import torch
from transformers.modeling_bert import BertForMaskedLM
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer

tokenizer = BertJapaneseTokenizer.from_pretrained("bert/ja/BERT-base_mecab-ipadic-bpe-32k_whole-word-mask")
model = BertForMaskedLM.from_pretrained("bert/ja/BERT-base_mecab-ipadic-bpe-32k_whole-word-mask")
model.eval()

input_ids = tokenizer.encode(f"""
山田さんが{tokenizer.mask_token}を見たのはこれが初めてでした。巨大だった。
""", return_tensors="pt")

masked_index = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]

result = model(input_ids)
result = result[0][:, masked_index].topk(5).indices.tolist()[0]

for r in result:
    output = input_ids[0].tolist()
    output[masked_index] = r
    print(tokenizer.decode(output))

[CLS] 山田さんが 顔 を見たのはこれが初めてでした 。 巨大だった 。 [SEP]
[CLS] 山田さんが 面倒 を見たのはこれが初めてでした 。 巨大だった 。 [SEP]
[CLS] 山田さんが 目 を見たのはこれが初めてでした 。 巨大だった 。 [SEP]
[CLS] 山田さんが 夢 を見たのはこれが初めてでした 。 巨大だった 。 [SEP]
[CLS] 山田さんが 姿 を見たのはこれが初めてでした 。 巨大だった 。 [SEP]


In [5]:
input_ids.shape

torch.Size([1, 23])

In [5]:
text = """ 日曜日の16時頃に訪問。 お店の前は通った事がありましたが 今回やっと初訪問。

この日はセールをやっていたようで
いつも外から眺めている印象よりお客さんが多いですね。

様々な種類のお肉が売っておりましたが
既に加工していたり揚げていたりする商品もあったので
そちらを中心に物色します。

コロッケやメンチカツなどもありましたが
唐揚げと串揚げを購入。

唐揚げはグラム数で計り売りで2個で95円でした。
串揚げは150円でした。

その日の夜に特に温めずに頂きましたが
唐揚げは温めなくてもジューシーな味わいで
個人的には非常に好みの味。

串揚げは中に玉葱と挽肉が入っておりましたが
中身はほとんど玉葱のみで、こちらは正直口には合わなかったですね。

それでも唐揚げはさすがお肉屋さんの味といった感想でした。

ご馳走様でした。
"""
tokenizer.encode(text, return_tensors='pt').shape

torch.Size([1, 240])

In [6]:
tokenizer = BertJapaneseTokenizer.from_pretrained("bert/ja/BERT-base_mecab-ipadic-bpe-32k_whole-word-mask")
model = BertForMaskedLM.from_pretrained("bert/ja/BERT-base_mecab-ipadic-bpe-32k_whole-word-mask")

input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
with torch.no_grad():
    last_hidden_states = model(input_ids)[0]

In [8]:
last_hidden_states.shape

torch.Size([1, 240, 32000])

In [21]:
short_text = "こんにちは"
input_ids = torch.tensor([tokenizer.encode(short_text, add_special_tokens=False)])
with torch.no_grad():
#     last_hidden_states = model(input_ids)[0]
#     second_layer = model(input_ids)[-2]
    all_layer = model(input_ids)

In [22]:
all_layer[0].shape

torch.Size([1, 3, 32000])

In [24]:
all_layer[0][0]

tensor([[-6.6648,  3.5317, -5.0330,  ..., -5.0268, -8.2822, -3.7546],
        [-6.6670,  4.8843, -1.5880,  ..., -7.1451, -5.1611, -5.6547],
        [-6.7644,  5.2466, -0.7674,  ..., -6.9232, -5.4712, -5.9564]])

In [25]:
tokenizer = BertJapaneseTokenizer.from_pretrained("bert-base-japanese-whole-word-masking")
model = BertForMaskedLM.from_pretrained("bert-base-japanese-whole-word-masking")

input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
with torch.no_grad():
    last_hidden_states = model(input_ids)[0]

ModuleNotFoundError: No module named 'MeCab'

In [1]:
# MeCabをimportしないとno module named mecabでエラー
# no vocab text はタイプミスの可能性。
# キャッシュされる。
# https://qiita.com/nekoumei/items/7b911c61324f16c43e7e
import MeCab
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('bert-base-japanese-whole-word-masking')
tokenizer.tokenize('お腹が痛いので遅れます。')

['お', '##腹', 'が', '痛', '##い', 'ので', '遅れ', 'ます', '。']

In [30]:
import sys
import MeCab

mecab = MeCab.Tagger("-Ochasen")

print(mecab.parse("すもももももももものうち"))

すもも	スモモ	すもも	名詞-一般		
も	モ	も	助詞-係助詞		
もも	モモ	もも	名詞-一般		
も	モ	も	助詞-係助詞		
もも	モモ	もも	名詞-一般		
の	ノ	の	助詞-連体化		
うち	ウチ	うち	名詞-非自立-副詞可能		
EOS



In [2]:
import torch
from transformers import BertModel
import MeCab
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('bert-base-japanese-whole-word-masking')
text = "テスト"
input_ids = torch.tensor([tokenizer.encode(text)]).unsqueeze(0)
print(input_ids)
model = BertModel.from_pretrained('bert-base-japanese-whole-word-masking')
print('loaded')
with torch.no_grad():
    output = model(input_ids)
    last_hiddden_states = outputs[0]
print(last_hiddden_states)

tensor([[[   2, 4609,    3]]])


KeyboardInterrupt: 

In [2]:
import torch
from transformers import BertTokenizer, BertModel



In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute")).unsqueeze(0)  # Batch size 1

KeyboardInterrupt: 

In [ ]:
outputs = model(input_ids)
last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple